In [1]:
import os

# Specify the path
path = "E:\Studies\Projects\YOLO (Hasnat)\class"

# List to store folder names
classes = []

# Loop through the directory
for folder in os.listdir(path):
    if os.path.isdir(os.path.join(path, folder)):
        classes.append(folder)

# Print the list
print("classes =", classes)


classes = ['Be Careful', 'Dont do that', 'I am Sorry', 'I Need Help', 'Need a bandage', 'Yes']


In [2]:
import os
import numpy as np
import torch
from glob import glob

# Base directory
base_dir = "E:\Studies\Projects\YOLO (Hasnat)\class"

# Classes
classes = ['Be Careful', 'Dont do that', 'I am Sorry', 'I Need Help', 'Need a bandage', 'Yes']

# Verification
for cl in classes:
    class_path = os.path.join(base_dir, cl)
    files = glob(os.path.join(class_path, '*.txt'))
    print(f"Class: {cl}, Number of Annotations: {len(files)}")

print("=================================================")
print("=================================================")

for cl in classes:
    class_path = os.path.join(base_dir, cl)
    files = glob(os.path.join(class_path, '*.jpg'))
    print(f"Class: {cl}, Number of Images: {len(files)}")

Class: Be Careful, Number of Annotations: 40
Class: Dont do that, Number of Annotations: 40
Class: I am Sorry, Number of Annotations: 40
Class: I Need Help, Number of Annotations: 40
Class: Need a bandage, Number of Annotations: 40
Class: Yes, Number of Annotations: 40
Class: Be Careful, Number of Images: 40
Class: Dont do that, Number of Images: 40
Class: I am Sorry, Number of Images: 40
Class: I Need Help, Number of Images: 40
Class: Need a bandage, Number of Images: 40
Class: Yes, Number of Images: 40


In [4]:
import shutil
import yaml

base_dir = "E:\Studies\Projects\YOLO (Hasnat)\class"

# Base directory for YOLOv8 datasets
yolov8_base_dir = "E:\Studies\Projects\YOLO (Hasnat)\yolov8_data"
os.makedirs(yolov8_base_dir, exist_ok=True)

# Directory for images and labels
images_dir = os.path.join(yolov8_base_dir, 'images')
labels_dir = os.path.join(yolov8_base_dir, 'labels')

# Create directories for training and validation images and labels
train_images_dir = os.path.join(images_dir, 'train')
train_labels_dir = os.path.join(labels_dir, 'train')
val_images_dir = os.path.join(images_dir, 'val')
val_labels_dir = os.path.join(labels_dir, 'val')

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Split data into train and validation sets
val_split = 0.2

for cl in classes:
    class_path = os.path.join(base_dir, cl)
    # Get all image files
    image_files = glob(os.path.join(class_path, '*.jpg'))
    # Shuffle the data before splitting
    np.random.shuffle(image_files)
    
    # Split the data
    val_size = int(len(image_files) * val_split)
    val_images = image_files[:val_size]
    train_images = image_files[val_size:]
    
    # Move the files to the corresponding training and validation directories
    for img_path in train_images:
        # Copy image to train directory
        shutil.copy(img_path, train_images_dir)
        # Copy corresponding label file to train labels directory
        txt_path = img_path.replace('.jpg', '.txt')
        shutil.copy(txt_path, train_labels_dir)
        
    for img_path in val_images:
        # Copy image to validation directory
        shutil.copy(img_path, val_images_dir)
        # Copy corresponding label file to validation labels directory
        txt_path = img_path.replace('.jpg', '.txt')
        shutil.copy(txt_path, val_labels_dir)

# Create dataset YAML file for YOLOv8
dataset_yaml = {
    'train': train_images_dir,
    'val': val_images_dir,
    'nc': len(classes),
    'names': classes
}

# Save it
with open(os.path.join(yolov8_base_dir, 'data.yaml'), 'w') as yaml_file:
    yaml.dump(dataset_yaml, yaml_file, default_flow_style=False)

print("Dataset prepared for YOLOv8.")


Dataset prepared for YOLOv8.


In [5]:
import os

train_labels_dir = "E:\Studies\Projects\YOLO (Hasnat)\yolov8_data\labels\train"
val_labels_dir = "E:\Studies\Projects\YOLO (Hasnat)\yolov8_data\labels\val"

classes = {
    "Be Careful": 0,
    "Dont do that": 1,
    "I am Sorry": 2,
    "I Need Help": 3,
    "Need a bandage": 4,
    "Yes": 5
}

def update_class_ids(label_dir):

    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):  # Ensure it's a text file
            file_path = os.path.join(label_dir, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()
            
            updated_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_name = parts[0]

                    for cl_name, cl_id in classes.items():
                        if cl_name in filename:  
                            parts[0] = str(cl_id)
                            break
                    updated_lines.append(" ".join(parts))
            
            with open(file_path, 'w') as file:
                file.write("\n".join(updated_lines) + "\n")

update_class_ids(train_labels_dir)
update_class_ids(val_labels_dir)

print("Class IDs updated for train and validation label files.")


Class IDs updated for train and validation label files.


In [11]:
from ultralytics import YOLO

dataset_path = "E:\\Studies\\Projects\\YOLO (Hasnat)\\yolov8_data\\dataset.yaml"

model = YOLO()

model.train(data=dataset_path, epochs=50, device='cpu')


Ultralytics YOLOv8.0.236 🚀 Python-3.8.0 torch-2.1.2+cu118 CPU (AMD Ryzen 5 5600 6-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=E:\Studies\Projects\YOLO (Hasnat)\yolov8_data\dataset.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

train: Scanning E:\Studies\Projects\YOLO (Hasnat)\yolov8_data\labels\train... 192 images, 0 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<00:00, 942.73it/s]


train: New cache created: E:\Studies\Projects\YOLO (Hasnat)\yolov8_data\labels\train.cache


val: Scanning E:\Studies\Projects\YOLO (Hasnat)\yolov8_data\labels\val... 48 images, 0 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<00:00, 603.63it/s]

val: New cache created: E:\Studies\Projects\YOLO (Hasnat)\yolov8_data\labels\val.cache


...

                   all         48         48      0.988      0.998      0.995       0.65

50 epochs completed in 0.840 hours.


Optimizer stripped from runs\detect\train6\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.2MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics YOLOv8.0.236 🚀 Python-3.8.0 torch-2.1.2+cu118 CPU (AMD Ryzen 5 5600 6-Core Processor)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.76s/it]


                   all         48         48      0.979          1      0.995      0.677
            Be Careful         48          8      0.986          1      0.995      0.577
          Dont do that         48          8      0.979          1      0.995      0.681
            I am Sorry         48          8      0.993          1      0.995      0.751
           I Need Help         48          8       0.97          1      0.995      0.798
        Need a bandage         48          8      0.972          1      0.995      0.696
                   Yes         48          8      0.974          1      0.995      0.557
Speed: 1.5ms preprocess, 63.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019B5BBE6FD0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("E:\\Studies\\Projects\\YOLO (Hasnat)\\runs\\detect\\train6\\weights\\best.pt").to('cpu')

results = model.predict(source = "0", show = True)


In [5]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
import supervision as sv
import sys

class ObjectDetection:

    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.device = 'cpu'  # Explicitly setting device to CPU
        print("Using Device: ", self.device)
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names
        self.box_annotator = sv.BoxAnnotator(color=sv.ColorPalette.default(), thickness=3, text_thickness=3, text_scale=1.5)
    

    def load_model(self):
        model = YOLO("E:\\Studies\\Projects\\YOLO (Hasnat)\\runs\\detect\\train6\\weights\\best.pt").to(self.device) 
        model.fuse()  # fuse model Conv2d + BatchNorm2d layers for speed
        return model


    def predict(self, frame):
        # Perform prediction using the YOLO model
        results = self.model(frame)
        return results
    

    def plot_bboxes(self, results, frame):
        # Extract detections and plot them
        detections = results[0].pred 
        # Iterate over detections and plot bounding boxes
        for *xyxy, conf, cls_id in detections:
            label = f"{self.CLASS_NAMES_DICT[int(cls_id)]} {conf:.2f}"
            cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
            cv2.putText(frame, label, (int(xyxy[0]), int(xyxy[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        return frame
    

    def __call__(self):
        # Run detection in loop and use 'q' key to break the loop
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Camera could not be accessed. Please ensure it is connected and not in use by another application.")
            sys.exit(1)

      
        while True:
            ret, frame = cap.read()
            assert ret
            
            results = self.predict(frame)
            frame = self.plot_bboxes(results, frame)
             
            cv2.imshow('YOLOv8 Detection', frame)
 
            # Break loop with 'q' key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
        
# Create an instance of the ObjectDetection and run it
detector = ObjectDetection(capture_index=1)
detector()


Using Device:  cpu
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs
Error: Camera could not be accessed. Please ensure it is connected and not in use by another application.


SystemExit: 1

c:\Users\tanji\anaconda3\envs\yolo\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
